## 1) Importações e Definição de Diretórios

In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import zip_longest
MICRODATA_FILES = {
    2021: Path("microdados/enem_2021/DADOS/MICRODADOS_ENEM_2021.csv"),
    2022: Path("microdados/enem_2022/DADOS/MICRODADOS_ENEM_2022.csv"),
    2023: Path("microdados/enem_2023/DADOS/MICRODADOS_ENEM_2023.csv"),
}
NA_VALUES = ["", " ", "NA", "N/A", "NULL", "-"]

## 2) Carregar microdados e concatenar

In [ ]:
frames = []
for ano, p in MICRODATA_FILES.items():
    if not p.exists():
        print('Arquivo não encontrado:', p)
        continue
    else:
        df = pd.read_csv(p, sep=';', decimal='.', encoding='iso-8859-1', dtype='string', na_values=NA_VALUES, low_memory=False)
    df['ANO_REFERENCIA'] = ano
    frames.append(df)
if not frames:
    raise RuntimeError('Nenhum arquivo carregado. Ajuste MICRODATA_FILES e rode novamente.')
dados = pd.concat(frames, ignore_index=True, sort=False)
print('Dados carregados:', dados.shape)

In [ ]:
dados.info()

## 3) Identificar colunas com Missing values (percentual)

In [ ]:

missing_pct = dados.isna().mean() * 100
display(missing_pct[missing_pct>0].sort_values(ascending=False).to_frame('missing_pct').head(50))

## 4) Tratamento para Missing Values
Uma quantidade considerável de missing values está relacionado ao não comparecimento nas provas. Para eliminar essas colunas, será considerado apenas os registros onde o aluno possui presença em todas as provas.

Além disso, todas as colunas que representam dados da escola do participante também possuem uma porcentagem alta de missing values, por conta disso, essas colunas serão desconsideradas.

Também iremos desconsiderar grande parte das perguntas do questionário socioeconômico, manteremos apenas perguntas que julgamos ser dados que podem ser usados para encontrar padrões no dataset.

Por fim, também iremos desconsiderar as colunas NU_INSCRICAO, TP_ANO_CONCLUIU, TP_ENSINO, CO_MUNICIPIO_PROVA, NO_MUNICIPIO_PROVA e também todas as colunas de Código do Tipo de Prova, que indicam qual cor de prova o participante realizou.

In [ ]:
required_presence = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT']
dados_limpos = dados.copy()
missing_presence = [c for c in required_presence if c not in dados_limpos.columns]
if missing_presence:
    print('Colunas de presença ausentes:', missing_presence)
else:
    presence_numeric = dados_limpos[required_presence].apply(pd.to_numeric, errors='coerce')
    mask = presence_numeric.eq(1).all(axis=1)
    dados_limpos = dados_limpos[mask].copy()
    print('Após filtrar presenças:', dados_limpos.shape)

In [ ]:
drop_columns = [
        'CO_MUNICIPIO_ESC',
        'TP_DEPENDENCIA_ADM_ESC',
        'NO_MUNICIPIO_ESC',
        'CO_UF_ESC',
        'SG_UF_ESC',
        'TP_SIT_FUNC_ESC',
        'TP_LOCALIZACAO_ESC',
    ]
cols_to_drop = [c for c in drop_columns if c in dados_limpos.columns]
if cols_to_drop:
    dados_limpos = dados_limpos.drop(columns=cols_to_drop)
    print('Colunas removidas:', cols_to_drop)
else:
    print('Colunas escolares já ausentes.')

In [ ]:
keep_q_cols = {'Q001', 'Q002', 'Q006', 'Q022', 'Q024', 'Q025'}
q_cols_to_drop = [c for c in dados_limpos.columns if c.startswith('Q0') and c not in keep_q_cols]
if q_cols_to_drop:
    dados_limpos = dados_limpos.drop(columns=q_cols_to_drop)
    print('Colunas Q0 removidas:', len(q_cols_to_drop))
else:
    print('Nenhuma coluna Q0 para remover.')

In [ ]:
extra_drop = ['NU_INSCRICAO', 'TP_ANO_CONCLUIU', 'TP_ENSINO', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA']
extra_cols = [c for c in extra_drop if c in dados_limpos.columns]
if extra_cols:
    dados_limpos = dados_limpos.drop(columns=extra_cols)
    print('Colunas extras removidas:', extra_cols)
else:
    print('Colunas extras já ausentes.')
co_prova_cols = [c for c in dados_limpos.columns if c.startswith('CO_PROVA')]
if co_prova_cols:
    dados_limpos = dados_limpos.drop(columns=co_prova_cols)
    print('Colunas CO_PROVA removidas:', len(co_prova_cols))

Com essas limpezas realizadas, ao refazer a busca de missing_values, apenas resta poucas respostas as perguntas do questionário socioeconômico mantidas que não foram respondidas.

In [ ]:
missing_pct = dados_limpos.isna().mean() * 100
display(missing_pct[missing_pct>0].sort_values(ascending=False).to_frame('missing_pct').head(50))

Quantidade real de registros restantes com missing values em cada coluna:

In [ ]:
dados_limpos.isna().sum()

Como sobrou apenas um registro com missing value, ele será desconsiderado

In [ ]:
dados_limpos[dados_limpos['Q006'].isna()]

In [ ]:
dados_limpos.dropna(subset=['Q006'], inplace=True)

In [ ]:
dados_limpos.isna().sum()

## 5) Enriquecimento
Foi decidido que iremos excluir as colunas de gabarito e respostas, e criar uma nova coluna para marcar as questões acertadas e erradas. No caso, as colunas de gabarito e respostas estão em formato string no padrão: 'abdebcedf*.a' sendo asterisco a dupla marcação e o . como em branco. A ideia da nova coluna para substituir será de colocar uma string apenas de 0 e 1, indicando erro e acerto. Com isso, também será criado uma nova coluna, que indica a porcentagem de acertos em determinada área do conhecimento.

In [ ]:
def respostas_para_boolean(resposta, gabarito):
    if not isinstance(resposta, str) or not isinstance(gabarito, str):
        return np.nan
    resp = resposta.upper()
    gab = gabarito.upper()
    bits = []
    for r, g in zip_longest(resp, gab, fillvalue=None):
        if r is None or g is None:
            continue
        if r in {'*', '.', ' '}:
            bits.append('0')
        elif r == g and g in {'A', 'B', 'C', 'D', 'E'}:
            bits.append('1')
        else:
            bits.append('0')
    return ''.join(bits) if bits else np.nan

def boolean_para_pct(bits):
    if not isinstance(bits, str) or not bits:
        return np.nan
    valid = [b for b in bits if b in {'0', '1'}]
    if not valid:
        return np.nan
    return (valid.count('1') / len(valid)) * 100

area_pairs = []
for gab_col in [c for c in dados_limpos.columns if c.startswith('TX_GABARITO_')]:
    area = gab_col.replace('TX_GABARITO_', '')
    resp_col = f'TX_RESPOSTAS_{area}'
    if resp_col in dados_limpos.columns:
        area_pairs.append((area, gab_col, resp_col))

if area_pairs:
    print('Áreas processadas:', [area for area, _, _ in area_pairs])
    for area, gab_col, resp_col in area_pairs:
        acertos_col = f'TX_ACERTOS_{area}'
        pct_col = f'PCT_ACERTO_{area}'
        dados_limpos[acertos_col] = [
            respostas_para_boolean(resp, gab)
            for resp, gab in zip(dados_limpos[resp_col], dados_limpos[gab_col])
        ]
        dados_limpos[pct_col] = dados_limpos[acertos_col].apply(boolean_para_pct)
    cols_to_drop = [col for _, gab_col, resp_col in area_pairs for col in (gab_col, resp_col)]
    dados_limpos.drop(columns=cols_to_drop, inplace=True)
    print('Colunas removidas:', cols_to_drop)
else:
    print('Nenhuma combinação de gabarito e respostas encontrada.')

Para facilitar a classificação de duas perguntas socioeconômicas, a Q022 e Q024, que são, respectivamente, de posse de celular e computador, iremos trocar os valores da resposta para apenas 0 e 1, 0 se não houver e 1 se houver 1 ou mais aparelhos.

In [ ]:
for col in ['Q022', 'Q024', 'Q025']:
    if col in dados_limpos.columns:
        dados_limpos[col] = dados_limpos[col].str.upper().map({'A': 0, 'B': 1, 'C': 1, 'D': 1, 'E': 1})
        print(f'{col} convertida para binário.')

In [ ]:
dados_limpos.head()

## 6) Padronização de Dados
Foi realizada a padronização dos tipos de dados do conjunto do ENEM, convertendo as variáveis categóricas (Q001, Q002, Q006 e TX_ACERTOS_*) para string e as demais colunas para valores numéricos. Além disso, os números decimais foram arredondados para duas casas decimais, garantindo consistência e melhor legibilidade dos dados.

In [ ]:
dados_limpos.info()

In [ ]:
cols_str = [
    'TX_ACERTOS_CN', 'TX_ACERTOS_CH', 'TX_ACERTOS_LC', 'TX_ACERTOS_MT',
    'Q001', 'Q002', 'Q006', 'TP_SEXO', 'SG_UF_PROVA'
]

for col in cols_str:
    if col in dados_limpos.columns:
        dados_limpos[col] = dados_limpos[col].astype(str)

cols_num = [col for col in dados_limpos.columns if col not in cols_str]

for col in cols_num:
    dados_limpos[col] = pd.to_numeric(dados_limpos[col], errors='coerce')

for col in cols_num:
    if pd.api.types.is_float_dtype(dados_limpos[col]):
        dados_limpos[col] = dados_limpos[col].round(2)

In [ ]:
dados_limpos.info()

In [ ]:
dados_limpos.head()

## 7) Outliers

In [ ]:
colunas_notas = [
    "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT",
    "NU_NOTA_COMP1", "NU_NOTA_COMP2", "NU_NOTA_COMP3",
    "NU_NOTA_COMP4", "NU_NOTA_COMP5", "NU_NOTA_REDACAO"
]

colunas_pct = ["PCT_ACERTO_CN", "PCT_ACERTO_CH", "PCT_ACERTO_LC", "PCT_ACERTO_MT"]

colunas_analise = colunas_notas + colunas_pct

df = dados_limpos.copy()

colunas_existentes = [c for c in colunas_analise if c in df.columns]

def detectar_outliers_iqr(df, coluna):
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    outliers = df[(df[coluna] < limite_inferior) | (df[coluna] > limite_superior)]
    return outliers, limite_inferior, limite_superior

resumo_outliers = []

for coluna in colunas_existentes:
    outliers, li, ls = detectar_outliers_iqr(df, coluna)
    resumo_outliers.append({
        "Coluna": coluna,
        "Qtd_Outliers": len(outliers),
        "Limite_Inferior": round(li, 2),
        "Limite_Superior": round(ls, 2)
    })
    
    plt.figure(figsize=(6, 4))
    sns.boxplot(data=df, y=coluna, color="skyblue")
    plt.title(f"Boxplot - {coluna}")
    plt.xlabel("")
    plt.ylabel("Valor")
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

resumo_df = pd.DataFrame(resumo_outliers)
print("Resumo de Outliers por Coluna:")
display(resumo_df.sort_values(by="Qtd_Outliers", ascending=False))


## 8) Exportação
Exportação dos dados limpos para um csv

In [50]:
output_path = Path('dados_limpos.csv')
dados_limpos.to_csv(output_path, index=False, encoding='utf-8')
print(f'Arquivo salvo em: {output_path.resolve()}')

Arquivo salvo em: D:\VsCode\CiencDados\EntregFinal\Etapa2e3\analise-enem\dados_limpos.csv
